In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment
import random
import math
import time
from genetic_matching import genetic_matching_with_elite

Daten laden und vorbereiten

In [2]:
# Lade die CSV-Dateien mit Personen, Teilaufgaben und Projekten
personen_df = pd.read_csv("../data/personen.csv")
teilaufgaben_df = pd.read_csv("../data/teilaufgaben.csv")

In [3]:
# Wandle die Kompetenzen in eine Liste um, um Filterung zu erleichtern
personen_df["kompetenzen_liste"] = personen_df["kompetenzen"].str.split(r",\s*")

print(f"Loaded: {len(personen_df)} people, {len(teilaufgaben_df)} tasks")
print(f"Total workload: {teilaufgaben_df['aufwand'].sum():.1f} hours")

Loaded: 150 people, 8 tasks
Total workload: 19.5 hours


Initialisierung

In [4]:
# Belegungstracker: Speichert die bereits verplante Kapazität je Person und Monat
belegungen = {}

In [5]:
# Liste zur Speicherung der Zuweisungsergebnisse
matching_ergebnisse = []

In [6]:
# Run Hungarian Algorithm
print("\n" + "="*50)
print("GENETIC ALGORITHM MATCHING")
print("="*50)


GENETIC ALGORITHM MATCHING


In [10]:
genetic_results, genetic_time = genetic_matching_with_elite(teilaufgaben_df, personen_df)


=== Enhanced Genetic Algorithm ===

Trying parameter set 1/3

=== Genetic Algorithm ===
Population size: 50
Generations: 100
Tasks: 8, People: 150
Generation 0: Best=-7597.4, Avg=-8577.2
Generation 20: Best=-7440.9, Avg=-7611.3
Generation 40: Best=-7038.4, Avg=-7188.2
Generation 60: Best=-7038.4, Avg=-7326.4
Generation 80: Best=-6584.5, Avg=-6754.3
Generation 99: Best=-6130.7, Avg=-6415.3
Execution time: 3.169 seconds
Best fitness: -6130.7
Found assignments: 8

Trying parameter set 2/3

=== Genetic Algorithm ===
Population size: 100
Generations: 150
Tasks: 8, People: 150
Generation 0: Best=-8000.0, Avg=-8691.2
Generation 20: Best=-7066.7, Avg=-7154.6
Generation 40: Best=-6130.7, Avg=-6291.0
Generation 60: Best=-5728.1, Avg=-5894.9
Generation 80: Best=-5728.1, Avg=-5860.8
Generation 100: Best=-5728.1, Avg=-5889.5
Generation 120: Best=-5728.1, Avg=-5939.3
Generation 140: Best=-5728.1, Avg=-5875.3
Generation 149: Best=-5728.1, Avg=-5894.9
Execution time: 11.374 seconds
Best fitness: -572

In [11]:
for result in genetic_results:
    matching_ergebnisse.append({
        "teilaufgabe_id": result["teilaufgabe_id"],
        "teilaufgabe": result["teilaufgabe"],
        "projekt_id": result["projekt_id"],
        "kompetenz": result["kompetenz"],
        "person_id": result["person_id"],
        "name": result["name"],
        "zugewiesener_aufwand": result["zugewiesener_aufwand"]
    })

print(f"\nGenetic Algorithm Results:")
print(f"Found {len(matching_ergebnisse)} assignments")
print(f"Execution time: {genetic_time:.3f} seconds")


Genetic Algorithm Results:
Found 24 assignments
Execution time: 25.224 seconds


In [12]:
# Display results summary
print(f"\n" + "="*50)
print("RESULTS SUMMARY")
print("="*50)
print(f"Total assignments: {len(matching_ergebnisse)}")
print(f"Total workload assigned: {sum(item['zugewiesener_aufwand'] for item in matching_ergebnisse):.1f} hours")
print(f"Total workload needed: {teilaufgaben_df['aufwand'].sum():.1f} hours")


RESULTS SUMMARY
Total assignments: 24
Total workload assigned: 58.5 hours
Total workload needed: 19.5 hours


In [13]:
# Show assignments by task
print(f"\nAssignments by task:")
for _, ta in teilaufgaben_df.iterrows():
    task_assignments = [item for item in matching_ergebnisse if item['teilaufgabe_id'] == ta['teilaufgabe_id']]
    assigned_workload = sum(item['zugewiesener_aufwand'] for item in task_assignments)
    print(f"  - {ta['bezeichnung']}: {assigned_workload:.1f}/{ta['aufwand']:.1f} hours assigned")


Assignments by task:
  - Entwurf CAD-Baugruppe: 9.0/3.0 hours assigned
  - Simulation FEM-Struktur: 7.5/2.5 hours assigned
  - Signalverarbeitung Sensor: 6.0/2.0 hours assigned
  - Entwicklung Python-Modul: 10.5/3.5 hours assigned
  - Statistische Auswertung: 6.0/2.0 hours assigned
  - Optische Analyse: 4.5/1.5 hours assigned
  - Datenbankmodellierung: 6.0/2.0 hours assigned
  - Thermodynamische Bewertung: 9.0/3.0 hours assigned


Ergebnisse speichern

In [14]:
# Save results
df_matching = pd.DataFrame(matching_ergebnisse)
df_matching.to_csv("matching_ergebnis.csv", index=False)
print(f"\nMatching abgeschlossen! Ergebnisse gespeichert in 'matching_ergebnis.csv'")


Matching abgeschlossen! Ergebnisse gespeichert in 'matching_ergebnis.csv'
